In [1]:
cd ..

/home/mehdi2277/Documents/HarveyMuddWork/Neural_Nets_Research/neural_nets_research


In [2]:
import torch.optim as optim
import torch.nn as nn
import torch
import matplotlib.pyplot as plt
import numpy as np

from neural_nets_library import training
from tree_to_sequence.tree_decoder import TreeDecoder
from tree_to_sequence.tree_to_tree_attention import TreeToTreeAttention
from tree_to_sequence.program_datasets import *
from functools import partial

In [3]:
torch.cuda.set_device(1)

In [4]:
use_cuda = False
image_width = 40
image_height = 32
one_hot = False
binarize_output = False
eos_token = False
long_base_case = True
output_as_seq = False
num_samples = 100

In [5]:
test_dset = ForLambdaDataset("ANC/AdditionalForDatasets/ForWithLevels/Easy-arbitraryForList.json", binarize_input=True, 
                                   binarize_output=binarize_output, eos_token=eos_token, one_hot=one_hot, 
                                   long_base_case=long_base_case, input_as_seq=True, 
                                   output_as_seq=output_as_seq, num_samples=100)


test_dset = [(torch.ones((1, 1, image_height, image_width)), output_image) for (input_image, output_image) in test_dset.program_pairs]


max_size = max([x[1].size() for x in test_dset])

In [6]:
class ImageEncoder(nn.Module):
    def __init__(self, nchannels, nhidden, num_layers, attention=True):
        super(ImageEncoder, self).__init__()
        self.core = nn.Sequential(CNN_Sequence_Extractor(nchannels), nn.LSTM(512, nhidden, num_layers, bidirectional=True))
        self.register_buffer('reverse_indices', torch.LongTensor(range(1, num_layers*2, 2)))
        print("reverse indices size is", self.reverse_indices.shape)
        self.attention = attention

    def forward(self, input, widths=None):
        output, (all_hiddens, _) = self.core(input)#, widths=widths) #TODO: Figure out what widths once did

        if widths is not None:
              output = nn.utils.rnn.pad_packed_sequence(output)

        reverse_hiddens = all_hiddens.index_select(0, self.reverse_indices) #TODO: does this need a gradient
        all_hiddens[-2:]
        print("all hiddens shape", all_hiddens.shape)
        print("reverse hiddens", reverse_hiddens.shape)
        
        if self.attention:
              return output.squeeze(1), reverse_hiddens.squeeze(1), torch.zeros(2,3)
        else:
              return reverse_hiddens
            
            
class CNN_Sequence_Extractor(nn.Module):
    def __init__(self, nchannels, leakyRelu=False):
        super(CNN_Sequence_Extractor, self).__init__()

        # Size of the kernel (image filter) for each convolutional layer.
        ks = [3, 3, 3, 3, 3, 3, 2]
        # Amount of zero-padding for each convoutional layer.
        ps = [1, 1, 1, 1, 1, 1, 0]
        # The stride for each convolutional layer. The list elements are of the form (height stride, width stride).
        ss = [(2,2), (2,2), (1,1), (2,1), (1,1), (2,1), (1,1)]
        # Number of channels in each convolutional layer.
        nm = [64, 128, 256, 256, 512, 512, 512]

        # Initializing the container for the modules that make up the neural network the neurel netowrk.
        cnn = nn.Sequential()

        # Represents a convolutional layer. The input paramter i signals that this is the ith convolutional layer. The user also has the option to set batchNormalization to True which will perform a batch normalization on the image after it has undergone a convoltuional pass. There is no output but this function adds the convolutional layer module created here to the sequential container, cnn.
        def convRelu(i, batchNormalization=False):
            nIn = nchannels if i == 0 else nm[i - 1]
            nOut = nm[i]
            cnn.add_module('conv{0}'.format(i),
                           nn.Conv2d(nIn, nOut, ks[i], ss[i], ps[i]))
            if batchNormalization:
                cnn.add_module('batchnorm{0}'.format(i), nn.BatchNorm2d(nOut))
            if leakyRelu:
                cnn.add_module('leaky_relu{0}'.format(i),
                               nn.LeakyReLU(0.2, inplace=True))
            else:
                cnn.add_module('relu{0}'.format(i), nn.ReLU(True))

        # Creating the 7 convolutional layers for the model.
        convRelu(0)
        convRelu(1)
        convRelu(2, True)
        convRelu(3)
        convRelu(4, True)
        convRelu(5)
        convRelu(6, True)

        self.cnn = cnn

    def forward(self, input, widths=None):
        print("input is", input)
        print("input size", input.shape)
        output = self.cnn(input)
        _, _, h, _ = output.size()
        print("output size", output.size())
        assert h == 1, "the height of conv must be 1"
        output = output.squeeze(2) # [b, c, w]
        output = output.permute(2, 0, 1) #[w, b, c]

        if widths is not None:
            sorted_widths, idx = widths.sort(descending=True)
            output = output.index_select(1, idx)
            output = nn.utils.pack_padded_sequence(output, sorted_widths / 4)

        return output

In [7]:
def reset_all_parameters_uniform(model, stdev):
    for param in model.parameters():
        nn.init.uniform_(param, -stdev, stdev)

In [8]:
eos_bonus = 1 if eos_token else 0
nclass = 50 # TODO: FIGURE THIS OUT
plot_every = 100
max_num_children = 4
hidden_size = 256
embedding_size = 25
alignment_size = 100
n_channels = 1
num_layers = 1 # TODO: Later consider making this work for num_layers > 1
max_size = 100 # TODO: FIGURE THIS OUT    
align_type = 1
    
encoder = ImageEncoder(n_channels, hidden_size, num_layers, attention=True)
decoder = TreeDecoder(embedding_size, hidden_size * 2, max_num_children, nclass=nclass)
program_model = TreeToTreeAttention(encoder, decoder, hidden_size * 2, embedding_size, nclass=nclass, max_size=max_size,
                                    alignment_size=alignment_size, align_type=align_type)
    
    
reset_all_parameters_uniform(program_model, 0.1)
decoder.initialize_forget_bias(3)


reverse indices size is torch.Size([1])
second case, dimensions are 512 512


In [9]:
if use_cuda:
    program_model = program_model.cuda()

In [10]:
optimizer = optim.Adam(program_model.parameters(), lr=0.005)
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True, patience=500, factor=0.8)

In [11]:
# Counts the number of matches between the prediction and target.
def count_matches(prediction, target):
    matches = 0
    if int(prediction.value) == int(target.value):
        matches += 1
    for i in range(min(len(target.children), len(prediction.children))):
        matches += count_matches(prediction.children[i], target.children[i])
    return matches

# Program accuracy (1 if completely correct, 0 otherwise)
def program_accuracy(prediction, target):
    if prediction.size() == count_matches(prediction, target) and \
       prediction.size() == target.size():
        return 1
    else:
        return 0

# Calculate validation accuracy (this could either be program or token accuracy)
def validation_criterion(prediction, target):
    return program_accuracy(prediction, target)

In [12]:

input = torch.zeros((100, 1, 32, 40))

attention_matrix , encoding, _ = encoder(input)
encoding = encoding.squeeze(0)

print(attention_matrix.shape)
print(encoding.shape)

input is tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        ...,


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,

In [13]:
best_model, train_plot_losses, validation_plot_losses = training.train_model_tree_to_tree(program_model, test_dset, 
                                 optimizer, lr_scheduler=None, num_epochs=5, plot_every=plot_every,
                                 batch_size=100, print_every=200, validation_criterion=validation_criterion,
                                 use_cuda=use_cuda)
    
    

Epoch 0/4
----------
input is tensor([[[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]]]])
input size torch.Size([1, 1, 32, 40])
output size torch.Size([1, 512, 1, 9])
all hiddens shape torch.Size([2, 1, 256])
reverse hiddens torch.Size([1, 1, 256])
dimesnsions of things the encoder returns: torch.Size([9, 512]) torch.Size([1, 256]) torch.Size([2, 3])
annotations shape!!! torch.Size([9, 512])
attention hidden values torch.Size([9, 512])
decoder hiddens torch.Size([1, 256])


RuntimeError: The size of tensor a (256) must match the size of tensor b (512) at non-singleton dimension 1

In [ ]:
xxxxx = torch.zeros((3,2))
print(xxxxx)
print(xxxxx.shape)

In [ ]:
plt.plot([x * plot_every for x in range(len(train_plot_losses))], train_plot_losses)
plt.show()

plt.plot([x * plot_every for x in range(len(validation_plot_losses))], validation_plot_losses)
plt.show()